###### https://arxiv.org/abs/2101.03087

In [12]:
import numpy as np 
import pandas as pd
import datetime

In [13]:
def read_data( filename,sheetname ):
    full_data = pd.read_excel( filename, sheetname )
    return full_data

def get_column_names( data ):
    column_row = data.iloc[3:4]
    column_names_dict = column_row.to_dict('records')
    values = list( column_names_dict[0].values() )
    values[0] = 'Date'
    return values

def create_numerical_subset( data, column_names ):
    subset = data.loc[5:]
    # rename the columns to the column names
    subset.columns = column_names
    return subset

def reformat_date( dataframe ):
    # create a new column wit the year
    dataframe['Year'] = dataframe['Date'].apply(lambda x: int( x.split('M')[0]) )
    # create a new column with the month
    dataframe['Month'] = dataframe['Date'].apply(lambda x: int( x.split('M')[1]))
    return dataframe

def get_subset_data( data, column_name, start_date, end_date ):
    # get the subset of the data that is between the start and end date
    subset = data[ (data['Year'] >= start_date) & (data['Year'] <= end_date)]
    subset = subset[['Year','Month',column_name]]
    return subset

def replace_dots( dataframe ):
    for idx, row in dataframe.iterrows():
        if type( row['Crude oil, WTI'] ) == str:
            row['Crude oil, WTI'] = row['Crude oil, Dubai']
            dataframe.loc[idx] = row
    return dataframe


def main():
    filename = 'CMO-Historical-Data-Monthly.xlsx'
    sheetname = 'Monthly Prices'
    dataframe = read_data(filename,sheetname)
    #print(dataframe)

    column_names = get_column_names(dataframe)
    print(column_names)

    start_date = 1960
    end_date = 2018

    #print( dataframe )
    numerical_data = create_numerical_subset(dataframe,column_names)
    numerical_data = reformat_date(numerical_data)
    #print( numerical_data )

    
    column_name = column_names[2]
    brent_data = get_subset_data(numerical_data,column_name,start_date,end_date)
    #print(brent_data)

    column_name = column_names[3]
    dubai_data = get_subset_data(numerical_data,column_name,start_date,end_date)
    #print(dubai_data)

    working_ds = pd.merge(brent_data,dubai_data,on=['Year','Month'])

    column_name = column_names[4]
    wti_data = get_subset_data(numerical_data,column_name,start_date,end_date)

    working_ds = pd.merge(working_ds,wti_data,on=['Year','Month'])

    working_ds = replace_dots(working_ds)

    # convert all data points to float
    return working_ds

In [14]:
working_ds = main()

['Date', 'Crude oil, average', 'Crude oil, Brent', 'Crude oil, Dubai', 'Crude oil, WTI', 'Coal, Australian', 'Coal, South African **', 'Natural gas, US', 'Natural gas, Europe', 'Liquefied natural gas, Japan', 'Natural gas index', 'Cocoa', 'Coffee, Arabica', 'Coffee, Robusta', 'Tea, avg 3 auctions', 'Tea, Colombo', 'Tea, Kolkata', 'Tea, Mombasa', 'Coconut oil', 'Groundnuts', 'Fish meal', 'Groundnut oil **', 'Palm oil', 'Palm kernel oil', 'Soybeans', 'Soybean oil', 'Soybean meal', 'Rapeseed oil', 'Sunflower oil', 'Barley', 'Maize', 'Sorghum', 'Rice, Thai 5% ', 'Rice, Thai 25% ', 'Rice, Thai A.1', 'Rice, Viet Namese 5%', 'Wheat, US SRW', 'Wheat, US HRW', 'Banana, Europe', 'Banana, US', 'Orange', 'Beef **', 'Chicken **', 'Lamb **', 'Shrimps, Mexican', 'Sugar, EU', 'Sugar, US', 'Sugar, world', 'Tobacco, US import u.v.', 'Logs, Cameroon', 'Logs, Malaysian', 'Sawnwood, Cameroon', 'Sawnwood, Malaysian', 'Plywood', 'Cotton, A Index', 'Rubber, TSR20 **', 'Rubber, RSS3', 'Phosphate rock', 'DAP', 

/tmp/ipykernel_1335928/2821151272.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Year'] = dataframe['Date'].apply(lambda x: int( x.split('M')[0]) )
/tmp/ipykernel_1335928/2821151272.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Month'] = dataframe['Date'].apply(lambda x: int( x.split('M')[1]))


In [15]:
working_ds

,Year,Month,"Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI"
0,1960,1,1.63,1.63,1.63
1,1960,2,1.63,1.63,1.63
2,1960,3,1.63,1.63,1.63
3,1960,4,1.63,1.63,1.63
4,1960,5,1.63,1.63,1.63
...,...,...,...,...,...
703,2018,8,73.13,72.13,67.99
704,2018,9,78.86,77.02,70.21
705,2018,10,80.47,78.96,70.75
706,2018,11,65.17,65.11,56.67


In [16]:
def get_stats(dataframe):
    stats_dict = {}

    # get the sum of the three columns 
    stats_dict['sum_brent'] = dataframe['Crude oil, Brent'].sum()
    stats_dict['mean_brent'] = dataframe['Crude oil, Brent'].mean()
    stats_dict['brent_max'] = dataframe['Crude oil, Brent'].max()
    stats_dict['brent_min'] = dataframe['Crude oil, Brent'].min()

    stats_dict['sum_dubai'] = dataframe['Crude oil, Dubai'].sum()
    stats_dict['mean_dubai'] = dataframe['Crude oil, Dubai'].mean()
    stats_dict['dubai_max'] = dataframe['Crude oil, Dubai'].max()
    stats_dict['dubai_min'] = dataframe['Crude oil, Dubai'].min()

    stats_dict['sum_wti'] = dataframe['Crude oil, WTI'].sum()
    stats_dict['mean_wti'] = dataframe['Crude oil, WTI'].mean()
    stats_dict['wti_max'] = dataframe['Crude oil, WTI'].max()
    stats_dict['wti_min'] = dataframe['Crude oil, WTI'].min()

    # get the mean of three columns
    stats_dict['mean_brent'] = dataframe['Crude oil, Brent'].mean()
    stats_dict['mean_dubai'] = dataframe['Crude oil, Dubai'].mean()
    stats_dict['mean_wti'] = dataframe['Crude oil, WTI'].mean()
    # get the average
    stats_dict['average'] = (stats_dict['mean_brent'] + stats_dict['mean_dubai'] + stats_dict['mean_wti']) / 3

    # median
    stats_dict['median_brent'] = dataframe['Crude oil, Brent'].median()
    stats_dict['median_dubai'] = dataframe['Crude oil, Dubai'].median()
    stats_dict['median_wti'] = dataframe['Crude oil, WTI'].median()
    # get the average
    stats_dict['median'] = (stats_dict['median_brent'] + stats_dict['median_dubai'] + stats_dict['median_wti']) / 3

    # get the standard deviation
    stats_dict['std_brent'] = dataframe['Crude oil, Brent'].std()
    stats_dict['std_dubai'] = dataframe['Crude oil, Dubai'].std()
    stats_dict['std_wti'] = dataframe['Crude oil, WTI'].std()
    # get the average
    stats_dict['std'] = (stats_dict['std_brent'] + stats_dict['std_dubai'] + stats_dict['std_wti']) / 3

    # get the average max
    stats_dict['average_max'] = (stats_dict['brent_max'] + stats_dict['dubai_max'] + stats_dict['wti_max']) / 3

    # get the average min
    stats_dict['average_min'] = (stats_dict['brent_min'] + stats_dict['dubai_min'] + stats_dict['wti_min']) / 3


    return stats_dict

In [17]:
stats_dict = get_stats(working_ds)
stats_dict

{'sum_brent': 21420.86776056653,
 'mean_brent': 30.255462938653288,
 'brent_max': 133.87304347826,
 'brent_min': 1.21,
 'sum_dubai': 20361.666205258804,
 'mean_dubai': 28.75941554415085,
 'dubai_max': 131.2247826087,
 'dubai_min': 1.21,
 'sum_wti': 20943.748973875896,
 'mean_wti': 29.58156634728234,
 'wti_max': 133.92714285714,
 'wti_min': 1.21,
 'average': 29.532148276695494,
 'median_brent': 19.174999999999997,
 'median_dubai': 17.27,
 'median_wti': 20.35,
 'median': 18.931666666666665,
 'std_brent': 30.90845036879654,
 'std_dubai': 29.896080417963994,
 'std_wti': 28.55992079063784,
 'std': 29.788150525799455,
 'average_max': 133.00832298136666,
 'average_min': 1.21}

###### Training and test datasets construction for LSTM modeling

In [18]:
def sliding_window(dataframe, mu, tau, train_ratio=0.7):
    # Calculate the number of training examples
    m = dataframe.shape[0]
    mtrain = int(m * train_ratio)
    mtrain = mtrain - mu
    
    # Initialize lists to hold the training and test data
    X_train, y_train = [], []
    X_test, y_test = [], []
    
    # Generate training data
    for i in range(mtrain):
        X_train.append(dataframe.iloc[i:i+mu].values)
        y_train.append(dataframe.iloc[i+mu:i+mu+tau].values)
    
    # Generate test data
    for i in range(mtrain, m - mu):
        X_test.append(dataframe.iloc[i:i+mu].values)
        y_test.append(dataframe.iloc[i+mu:i+mu+tau].values)
    
    # Convert lists to numpy arrays and reshape
    X_train = np.array(X_train)
    y_train = np.array(y_train).reshape(-1, tau, dataframe.shape[1])
    X_test = np.array(X_test)
    y_test = np.array(y_test).reshape(-1, tau, dataframe.shape[1])
    
    return X_train, y_train, X_test, y_test

mu = 3
tau = 1
mtrain = 10

X_train, y_train, X_test, y_test = sliding_window(working_ds, mu, tau)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(492, 3, 5)
(492, 1, 5)
(213, 3, 5)
(213, 1, 5)


In [19]:
print( X_train )

[[[1960 1 1.63000011444 1.63000011444 1.63000011444]
  [1960 2 1.63000011444 1.63000011444 1.63000011444]
  [1960 3 1.63000011444 1.63000011444 1.63000011444]]

 [[1960 2 1.63000011444 1.63000011444 1.63000011444]
  [1960 3 1.63000011444 1.63000011444 1.63000011444]
  [1960 4 1.63000011444 1.63000011444 1.63000011444]]

 [[1960 3 1.63000011444 1.63000011444 1.63000011444]
  [1960 4 1.63000011444 1.63000011444 1.63000011444]
  [1960 5 1.63000011444 1.63000011444 1.63000011444]]

 ...

 [[2000 10 30.9323 30.2223 33.0459]
  [2000 11 32.52409090909 30.10090909091 34.368]
  [2000 12 25.1255 22.086 28.4025]]

 [[2000 11 32.52409090909 30.10090909091 34.368]
  [2000 12 25.1255 22.086 28.4025]
  [2001 1 25.63636363636 22.68545454545 29.55]]

 [[2000 12 25.1255 22.086 28.4025]
  [2001 1 25.63636363636 22.68545454545 29.55]
  [2001 2 27.406 24.742 29.5685]]]


###### Training and test datasets construction for LSTM modeling

###### LSTM hyperparameters for baseline model


 | Hyperparameter | Value |
 | --- | --- |
 Lookback | 10
 Dropout | 0.1
 Units | 50
 Epochs | 50
 Batch size | 32
 | --- | --- |

|Adam Hyperparameters | Value |
| --- | --- |
Learning rate | 0.001
Beta 1 | 0.9
Beta 2 | 0.999
Epsilon | 1e-07



In [20]:
df = working_ds;

In [29]:
# Assuming df is your DataFrame
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
#df['Year'] = df['Date'].dt.year
#df['Month'] = df['Date'].dt.month
#df['Quarter'] = df['Date'].dt.quarter
#df['DayOfYear'] = df['Date'].dt.dayofyear
#df['WeekOfYear'] = df['Date'].dt.isocalendar().week
#df['DayOfWeek'] = df['Date'].dt.dayofweek
#df['IsMonthStart'] = df['Date'].dt.is_month_start
#df['IsMonthEnd'] = df['Date'].dt.is_month_end

In [30]:
for lag in range(1, 13):  # Create lag features for the past 12 months
    df[f'Brent_Lag_{lag}'] = df['Crude oil, Brent'].shift(lag)
    df[f'Dubai_Lag_{lag}'] = df['Crude oil, Dubai'].shift(lag)
    df[f'WTI_Lag_{lag}'] = df['Crude oil, WTI'].shift(lag)

In [31]:
"""
df['Brent_Rolling_Mean_3'] = df['Crude oil, Brent'].rolling(window=3).mean()
df['Brent_Rolling_Std_3'] = df['Crude oil, Brent'].rolling(window=3).std()
df['Dubai_Rolling_Mean_3'] = df['Crude oil, Dubai'].rolling(window=3).mean()
df['Dubai_Rolling_Std_3'] = df['Crude oil, Dubai'].rolling(window=3).std()
df['WTI_Rolling_Mean_3'] = df['Crude oil, WTI'].rolling(window=3).mean()
df['WTI_Rolling_Std_3'] = df['Crude oil, WTI'].rolling(window=3).std()
"""

"\ndf['Brent_Rolling_Mean_3'] = df['Crude oil, Brent'].rolling(window=3).mean()\ndf['Brent_Rolling_Std_3'] = df['Crude oil, Brent'].rolling(window=3).std()\ndf['Dubai_Rolling_Mean_3'] = df['Crude oil, Dubai'].rolling(window=3).mean()\ndf['Dubai_Rolling_Std_3'] = df['Crude oil, Dubai'].rolling(window=3).std()\ndf['WTI_Rolling_Mean_3'] = df['Crude oil, WTI'].rolling(window=3).mean()\ndf['WTI_Rolling_Std_3'] = df['Crude oil, WTI'].rolling(window=3).std()\n"

In [32]:
"""
df['Brent_Diff'] = df['Crude oil, Brent'].diff()
df['Dubai_Diff'] = df['Crude oil, Dubai'].diff()
df['WTI_Diff'] = df['Crude oil, WTI'].diff()
"""

"\ndf['Brent_Diff'] = df['Crude oil, Brent'].diff()\ndf['Dubai_Diff'] = df['Crude oil, Dubai'].diff()\ndf['WTI_Diff'] = df['Crude oil, WTI'].diff()\n"

In [33]:
"""
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df['Crude oil, Brent'], model='additive', period=12)
df['Brent_Trend'] = result.trend
df['Brent_Seasonal'] = result.seasonal
df['Brent_Residual'] = result.resid

result = seasonal_decompose(df['Crude oil, Dubai'], model='additive', period=12)
df['Dubai_Trend'] = result.trend
df['Dubai_Seasonal'] = result.seasonal
df['Dubai_Residual'] = result.resid

result = seasonal_decompose(df['Crude oil, WTI'], model='additive', period=12)
df['WTI_Trend'] = result.trend
df['WTI_Seasonal'] = result.seasonal
df['WTI_Residual'] = result.resid
"""

"\nfrom statsmodels.tsa.seasonal import seasonal_decompose\n\nresult = seasonal_decompose(df['Crude oil, Brent'], model='additive', period=12)\ndf['Brent_Trend'] = result.trend\ndf['Brent_Seasonal'] = result.seasonal\ndf['Brent_Residual'] = result.resid\n\nresult = seasonal_decompose(df['Crude oil, Dubai'], model='additive', period=12)\ndf['Dubai_Trend'] = result.trend\ndf['Dubai_Seasonal'] = result.seasonal\ndf['Dubai_Residual'] = result.resid\n\nresult = seasonal_decompose(df['Crude oil, WTI'], model='additive', period=12)\ndf['WTI_Trend'] = result.trend\ndf['WTI_Seasonal'] = result.seasonal\ndf['WTI_Residual'] = result.resid\n"

In [34]:
#df['Brent_EMA_3'] = df['Crude oil, Brent'].ewm(span=3, adjust=False).mean()
#df['Dubai_EMA_3'] = df['Crude oil, Dubai'].ewm(span=3, adjust=False).mean()
#df['WTI_EMA_3'] = df['Crude oil, WTI'].ewm(span=3, adjust=False).mean()

In [35]:
df

,Year,Month,"Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI",Brent_Lag_1,Dubai_Lag_1,WTI_Lag_1,Brent_Lag_2,Dubai_Lag_2,...,Brent_Lag_10,Dubai_Lag_10,WTI_Lag_10,Brent_Lag_11,Dubai_Lag_11,WTI_Lag_11,Brent_Lag_12,Dubai_Lag_12,WTI_Lag_12,Date
0,1960,1,1.63,1.63,1.63,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1960-01-01
1,1960,2,1.63,1.63,1.63,1.63,1.63,1.63,None,None,...,None,None,None,None,None,None,None,None,None,1960-02-01
2,1960,3,1.63,1.63,1.63,1.63,1.63,1.63,1.63,1.63,...,None,None,None,None,None,None,None,None,None,1960-03-01
3,1960,4,1.63,1.63,1.63,1.63,1.63,1.63,1.63,1.63,...,None,None,None,None,None,None,None,None,None,1960-04-01
4,1960,5,1.63,1.63,1.63,1.63,1.63,1.63,1.63,1.63,...,None,None,None,None,None,None,None,None,None,1960-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2018,8,73.13,72.13,67.99,74.44,72.72,70.84,75.19,73.22,...,57.62,55.58,51.56,55.16,53.86,49.83,51.37,50.43,48.03,2018-08-01
704,2018,9,78.86,77.02,70.21,73.13,72.13,67.99,74.44,72.72,...,62.57,60.58,56.65,57.62,55.58,51.56,55.16,53.86,49.83,2018-09-01
705,2018,10,80.47,78.96,70.75,78.86,77.02,70.21,73.13,72.13,...,64.21,61.41,57.94,62.57,60.58,56.65,57.62,55.58,51.56,2018-10-01
706,2018,11,65.17,65.11,56.67,80.47,78.96,70.75,78.86,77.02,...,68.99,66.02,63.67,64.21,61.41,57.94,62.57,60.58,56.65,2018-11-01


In [36]:
from prophet import Prophet


# Prepare the data for Prophet
df_prophet = df[['Date', 'Crude oil, Brent']].rename(columns={'Date': 'ds', 'Crude oil, Brent': 'y'})
df_prophet['Dubai'] = df['Crude oil, Dubai']
df_prophet['WTI'] = df['Crude oil, WTI']

# Initialize and fit the model
model = Prophet()
model.add_regressor('Dubai')
model.add_regressor('WTI')
model.fit(df_prophet)

# Make future predictions
future = model.make_future_dataframe(periods=12, freq='M')
future['Dubai'] = df['Crude oil, Dubai']
future['WTI'] = df['Crude oil, WTI']
forecast = model.predict(future)

22:30:15 - cmdstanpy - INFO - Chain [1] start processing
22:30:15 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Found NaN in column 'Dubai'